In [1]:
import random

def generateBoard(cols, rows, mines):
    board = [[0 for x in range(cols)] for y in range(rows)]

    #generate mine locations
    minePos = random.sample(range(rows * cols), mines)
    for pos in minePos:
        row, col = divmod(pos, cols)
        board[row][col] = 'M'

    #count neighbors
    for row in range(rows):
        for col in range(cols):
            if board[row][col] == 'M':
                continue
            count = 0
            for rowShift in [-1, 0, 1]:
                for colShift in [-1, 0, 1]:
                    r= row + rowShift
                    c = col + colShift
                    if (0 <= r < rows) and (0 <= c < cols) and (board[r][c] == 'M'):
                        count += 1
            board[row][col] = count

    return board

def generatePlayArea(cols, rows):
    PlayArea = [['X' for x in range(cols)] for y in range(rows)]
    return PlayArea

# def reveal_cell(board, play_area, col, row):
#     if play_area[row][col] != 'X':
#         return False
#     if board[row][col] == 'M':
#         play_area[row][col] = 'M'
#         return True
#     elif board[row][col] > 0:
#         play_area[row][col] = str(board[row][col])
#     else:
#         play_area[row][col] = ' '
#         for row_shift in [-1, 0, 1]:
#             for col_shift in [-1, 0, 1]:
#                 r = row + row_shift
#                 c = col + col_shift
#                 if 0 <= r < len(board) and 0 <= c < len(board[0]):
#                     if play_area[r][c] == 'X':
#                         reveal_cell(board, play_area, c, r)
#     return False

def print_board(play_area):
    for row in play_area:
        print(" ".join(row))
    print()

# def check_win(board, play_area):
#     for row in range(len(board)):
#         for col in range(len(board[0])):
#             if board[row][col] != 'M' and play_area[row][col] == 'X':
#                 return False
#     return True

def checkSpot(board, col, row):
    result = board[row][col]
    return result

In [2]:
board = generateBoard(30,16,40)
playArea = generatePlayArea(30,16)

for row in board:
    print(row)
print("----------")
for row in playArea:
    print(row)

[0, 0, 0, 1, 'M', 1, 0, 1, 'M', 1, 0, 1, 'M', 2, 1, 0, 0, 0, 1, 2, 'M', 1, 0, 0, 0, 0, 1, 1, 2, 1]
[0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 2, 'M', 1, 0, 1, 1, 2, 'M', 3, 2, 0, 0, 0, 1, 2, 'M', 2, 'M']
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 'M', 2, 2, 'M', 1, 1, 1, 1, 1, 'M', 2, 2, 1]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 'M', 1, 1, 1, 1, 0, 0]
[0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 'M', 2, 1, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 1, 'M', 1, 1, 2, 2, 1, 0, 1, 1, 2, 'M', 1, 0, 0, 1, 1, 2, 1, 1, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 1, 1, 2, 2, 'M', 'M', 1, 0, 0, 0, 1, 1, 2, 1, 1, 1, 'M', 3, 'M', 2, 0, 0, 1, 1, 1]
[0, 0, 0, 0, 0, 0, 0, 1, 'M', 3, 2, 1, 0, 0, 0, 0, 0, 1, 'M', 1, 1, 1, 3, 'M', 2, 0, 0, 1, 'M', 1]
[0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 2, 2, 1, 0, 0, 0, 1, 2, 2, 1, 0, 1, 1, 1, 0, 1, 2, 2, 1]
[0, 1, 'M', 1, 0, 0, 0, 0, 0, 0, 1, 'M', 'M', 1, 0, 0, 0, 0, 1, 'M', 1, 0, 0, 0, 0, 0, 1, 'M', 2, 1]
[0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 

In [3]:
import copy
import numpy as np

def countNeighborUnknowns(playArea, col, row):
    unseenCount = 0
    flagCount = 0
    for colShift in [-1, 0, 1]:
        for rowShift in [-1, 0, 1]:
            curCol = col + colShift
            curRow = row + rowShift

            if (curRow >= 0 and curRow < len(playArea) and curCol >= 0 and curCol < len(playArea[0])):
                if(playArea[curRow][curCol] == 'X'):
                    unseenCount += 1
                if(playArea[curRow][curCol] == 'F'):
                    flagCount += 1

    return unseenCount, flagCount



def getFrontier(playArea):
    frontier = []
    for i in range(len(playArea)):
        for j in range(len(playArea[0])):
            unseenCount, flagCount = countNeighborUnknowns(playArea, j, i)
            if (isinstance(playArea[i][j], int) and unseenCount >= 1):

                #could append both unseen count or flag count here?
                frontier.append((j,i, unseenCount, flagCount))

    return frontier

def depthFirstSearch(board, playArea, col, row):
    if (row < 0 or row >= len(board) or col < 0 or col >= len(board[0]) or playArea[row][col] != 'X'):
        return

    playArea[row][col] = checkSpot(board, col, row)

    if playArea[row][col] == 'M':
        print("lost during dfs")
        return

    # if no nearby mines, check all nearby squares
    if playArea[row][col] == 0:
        for i in [-1, 0, 1]:
            for j in [-1, 0, 1]:
                depthFirstSearch(board, playArea, col+i, row+i)
    elif isinstance(playArea[row][col], int):
        # if it is a number, we need to backtrack?
        return

def solveCertainMoves(board, playArea):
    frontier = getFrontier(playArea)
    for (col, row, unseenCount, flagCount) in frontier:
        if (playArea[row][col] == 'X' or playArea[row][col] == 'F'):
            print("string in the frontier")
            continue
        if (playArea[row][col] - flagCount == unseenCount):
            for colShift in [-1, 0, 1]:
                for rowShift in [-1, 0, 1]:
                    curCol = col + colShift
                    curRow = row + rowShift

                    if (curRow >= 0 and curRow < len(playArea) and curCol >= 0 and curCol < len(playArea[0]) and playArea[curRow][curCol] == 'X'):
                        playArea[curRow][curCol] = 'F'
        elif (playArea[row][col] - flagCount == 0 and unseenCount > 0):
            # if all mines have been seen, but there are still more neighbor squares to explore
            for colShift in [-1, 0, 1]:
                for rowShift in [-1, 0, 1]:
                    curCol = col + colShift
                    curRow = row + rowShift

                    if (curRow >= 0 and curRow < len(playArea) and curCol >= 0 and curCol < len(playArea[0]) and playArea[curRow][curCol] == 'X'):
                        depthFirstSearch(board, playArea, curCol, curRow)

def guess(board, playArea):
    frontier = getFrontier(playArea)
    bestOdd = 1.1
    bestCellRow = -1
    bestCellCol = -1

    #iterate across frontier, getting cell with best odds and guess that one of it's adjacent neighbors is not a mine
    for (col, row, unseenCount, flagCount) in frontier:
        curVal = playArea[row][col]
        remaining = curVal - flagCount
        curOdds = remaining / unseenCount
        if (curOdds < bestOdd):
            bestOdd = curOdds
            bestCellCol = col
            bestCellRow = row
    
    #get an adjacent unexplored
    for colShift in [-1, 0, 1]:
        for rowShift in [-1, 0, 1]:
            curCol = bestCellCol + colShift
            curRow = bestCellRow + rowShift
            if (curRow >= 0 and curRow < len(playArea) and curCol >= 0 and curCol < len(playArea[0]) and playArea[curRow][curCol] == 'X'):
                guessResult = checkSpot(board, curCol, curRow)
                print(f"Made guess with odds: {bestOdd:.4f} on cell: ({curCol}, {curRow})")
                playArea[curRow][curCol] = guessResult
                if (guessResult == 'M'):
                    print("Lost in guess function")
                    return False
                else:
                    print("Guess succeeds")
                    return True
    
    print("didn't find adjacent unexplored during guess")
    return False


def checkIfDone(playArea):
    for row in range(len(playArea)):
        for col in range(len(playArea[0])):
            if (playArea[row][col] == 'X' or playArea[row][col] == 'M'):
                return False
    return True



def solve(board, playArea):
    isAlive = True
    pastState = copy.deepcopy(playArea)
    row = -1
    col = -1
    foundStart = False
    
    for i in range(len(board)):
        for j in range(len(board[0])):
            cellVal = checkSpot(board, col, row)

            if cellVal == 'M':
                #print("lost during solve")
                continue
            else:
                row = i
                col = j
                foundStart = True
                break
        if foundStart:
            break
    #explore zeros
    depthFirstSearch(board, playArea, col, row)

    while (not checkIfDone(playArea)):
        #create frontier
        #solve certain moves
        solveCertainMoves(board, playArea)

        #check if past state is same as cur state
        # if so, we need to guess
        np_playArea = np.array(playArea)
        np_pastState = np.array(pastState)
        if (np.all(np_playArea == np_pastState)):
            isAlive = guess(board, playArea)

        pastState = copy.deepcopy(playArea)

        if (not isAlive):
            print("dead")
            return False
    
    print("win")
    return True


board = generateBoard(30,16,70)
playArea = generatePlayArea(30,16)

for row in playArea:
    print(row)

solve(board, playArea)

print("----------------")

for row in playArea:
    print(row)

['X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X']
['X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X']
['X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X']
['X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X']
['X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X']
['X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X']
['X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X'

In [4]:
winCount = 0
numGames = 100

for i in range(numGames):
    board = generateBoard(30,16,40)
    playArea = generatePlayArea(30,16)
    result = solve(board, playArea)
    if(result):
        winCount += 1

print(f"Result after {numGames}: {winCount}")

win
lost during dfs
didn't find adjacent unexplored during guess
dead
win
Made guess with odds: 1.1000 on cell: (0, 0)
Guess succeeds
Made guess with odds: 0.2000 on cell: (0, 3)
Lost in guess function
dead
Made guess with odds: 0.3333 on cell: (0, 1)
Guess succeeds
Made guess with odds: 0.2500 on cell: (0, 2)
Guess succeeds
Made guess with odds: 0.2500 on cell: (0, 3)
Guess succeeds
win
win
win
win
Made guess with odds: 0.3333 on cell: (0, 1)
Lost in guess function
dead
win
Made guess with odds: 0.3333 on cell: (0, 1)
Lost in guess function
dead
Made guess with odds: 0.3333 on cell: (28, 13)
Guess succeeds
Made guess with odds: 0.3333 on cell: (27, 12)
Lost in guess function
dead
win
win
win
Made guess with odds: 0.3333 on cell: (15, 0)
Guess succeeds
win
win
lost during dfs
didn't find adjacent unexplored during guess
dead
lost during dfs
didn't find adjacent unexplored during guess
dead
Made guess with odds: 0.3333 on cell: (0, 3)
Lost in guess function
dead
win
Made guess with odds